In [1]:
!pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
!pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
!pip install ipython-sql

  Using cached ibm_db-3.1.0-py3-none-any.whl
  Using cached ibm_db_sa-0.3.3-py3-none-any.whl
  Using cached SQLAlchemy-2.0.16-cp39-cp39-win_amd64.whl (2.0 MB)
  Using cached typing_extensions-4.6.3-py3-none-any.whl (31 kB)
  Using cached greenlet-2.0.2-cp39-cp39-win_amd64.whl (192 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.6.3
    Uninstalling typing-extensions-4.6.3:
      Successfully uninstalled typing-extensions-4.6.3
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.9
    Uninstalling SQLAlchemy-1.3.9:
      Successfully uninstalled SQLAlchemy-1.3.9
  Attempting uninstall: ibm-db-sa
    Found existing installation: ibm-db-sa 0.4.0
    Uninstalling ibm-db-sa-0.4.0:
      Successfully uninstalled ibm-db-sa-0.4.0
  Attempting uninstall: ib

Found existing installation: SQLAlchemy 2.0.16
Uninstalling SQLAlchemy-2.0.16:
  Successfully uninstalled SQLAlchemy-2.0.16


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.24 which is incompatible.


  Using cached SQLAlchemy-1.3.24-cp39-cp39-win_amd64.whl (1.2 MB)
  Using cached prettytable-0.7.2-py3-none-any.whl
  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.8.0
    Uninstalling prettytable-3.8.0:
      Successfully uninstalled prettytable-3.8.0


  Attempting uninstall: ipython-sql
    Found existing installation: ipython-sql 0.5.0
    Uninstalling ipython-sql-0.5.0:
      Successfully uninstalled ipython-sql-0.5.0


In [2]:
import ibm_db

In [3]:
#Db2 Service Credentials
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"          
dsn_hostname = "3883e7e4-18f5-4afe-be8c-fa31c41761d2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"           
dsn_port = "31498"                  
dsn_protocol = "TCPIP"        
dsn_uid = "lfp94968"                
dsn_pwd = "zU0Wh8VKm32LrH3p"                
dsn_security = "SSL"              

In [4]:

#Create database connection
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  bludb as user:  lfp94968 on host:  3883e7e4-18f5-4afe-be8c-fa31c41761d2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud



## Problem 1: Find the total number of crimes recorded in the CRIME table.

In [5]:
#Construct the query that retrieves all rows from the Chicago Crime data table
selectQuery = "select COUNT(*) AS TOTAL_NUMBER_OF_CRIMES_RECORDED from CHICAGO_CRIME_DATA"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

ibm_db.fetch_assoc(selectStmt)

{'TOTAL_NUMBER_OF_CRIMES_RECORDED': 533}

## Problem 2: List community areas with per capita income less than 11000.

In [6]:
#Construct the query that lists all rows with per capita income less than 11000
selectQuery = "select COMMUNITY_AREA_NAME,PER_CAPITA_INCOME from CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000 "

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'COMMUNITY_AREA_NAME': 'West Garfield Park', 'PER_CAPITA_INCOME': 10934}
{'COMMUNITY_AREA_NAME': 'South Lawndale', 'PER_CAPITA_INCOME': 10402}
{'COMMUNITY_AREA_NAME': 'Fuller Park', 'PER_CAPITA_INCOME': 10432}
{'COMMUNITY_AREA_NAME': 'Riverdale', 'PER_CAPITA_INCOME': 8201}


## Problem 3: List all case numbers for crimes involving minors?

In [7]:
#Construct the query that lists all the case numbers involving minors from the Chicago Crime data table
selectQuery = "select CASE_NUMBER,DESCRIPTION from CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR%'"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'CASE_NUMBER': 'HL266884', 'DESCRIPTION': 'SELL/GIVE/DEL LIQUOR TO MINOR'}
{'CASE_NUMBER': 'HK238408', 'DESCRIPTION': 'ILLEGAL CONSUMPTION BY MINOR'}


## Problem 4: List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [8]:
#Construct the query that lists all the kidnapping involving a child from the Chicago Crime data table
selectQuery = """select PRIMARY_TYPE,DESCRIPTION from CHICAGO_CRIME_DATA 
WHERE PRIMARY_TYPE='KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'"""

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'PRIMARY_TYPE': 'KIDNAPPING', 'DESCRIPTION': 'CHILD ABDUCTION/STRANGER'}


## Problem 5: What kind of crimes were recorded at schools?

In [9]:
#Construct the query that lists the kind of crimes recorded at schools from the Chicago Crime data table
selectQuery = "select DISTINCT PRIMARY_TYPE from CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'PRIMARY_TYPE': 'ASSAULT'}
{'PRIMARY_TYPE': 'BATTERY'}
{'PRIMARY_TYPE': 'CRIMINAL DAMAGE'}
{'PRIMARY_TYPE': 'CRIMINAL TRESPASS'}
{'PRIMARY_TYPE': 'NARCOTICS'}
{'PRIMARY_TYPE': 'PUBLIC PEACE VIOLATION'}


## Problem 6: List the average safety score for all types of schools.

In [10]:
#Construct the query that lists the average safety score for all types of schools from the Chicago public schools data table
selectQuery = """select ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL,AVG(SAFETY_SCORE) AS AVERAGE_SAFETY_SCORE from CHICAGO_PUBLIC_SCHOOLS 
GROUP BY ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL"""

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL': 'ES', 'AVERAGE_SAFETY_SCORE': 49}
{'ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL': 'HS', 'AVERAGE_SAFETY_SCORE': 49}
{'ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL': 'MS', 'AVERAGE_SAFETY_SCORE': 48}


## Problem 7: List 5 community areas with highest % of households below poverty line.

In [11]:
#Construct the query that lists the community areas with highest % of households below poverty line from the census data table
selectQuery = """select COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY from CENSUS_DATA 
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"""

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'COMMUNITY_AREA_NAME': 'Riverdale', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '56.5'}
{'COMMUNITY_AREA_NAME': 'Fuller Park', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '51.2'}
{'COMMUNITY_AREA_NAME': 'Englewood', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '46.6'}
{'COMMUNITY_AREA_NAME': 'North Lawndale', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '43.1'}
{'COMMUNITY_AREA_NAME': 'East Garfield Park', 'PERCENT_HOUSEHOLDS_BELOW_POVERTY': '42.4'}


## Problem 8: Which community area(number) is most crime prone?

In [12]:
#Construct the query that shows community area with the most crime prone from the Chicago Crime data table
selectQuery = """select COMMUNITY_AREA_NUMBER,COUNT(*) AS CRIME_COUNT from CHICAGO_CRIME_DATA 
GROUP BY COMMUNITY_AREA_NUMBER ORDER BY CRIME_COUNT DESC LIMIT 1"""

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'COMMUNITY_AREA_NUMBER': 25, 'CRIME_COUNT': 43}


## Problem 9: Use a sub-query to find the name of the community area with highest hardship index.

In [13]:
#Construct the query that finds the name of the community area with highest hardship index
selectQuery = "select COMMUNITY_AREA_NAME,HARDSHIP_INDEX from CENSUS_DATA WHERE HARDSHIP_INDEX=(SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA) "

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'COMMUNITY_AREA_NAME': 'Riverdale', 'HARDSHIP_INDEX': 98}


## Problem 10: Use a sub-query to determine the Community Area Name with most number of crimes?

In [14]:
#Construct the query that determines the Community Area Name with most number of crimes from the Chicago Crime data table
selectQuery = """select c.COMMUNITY_AREA_NAME,d.COMMUNITY_AREA_NUMBER,d.CRIME_COUNT FROM CENSUS_DATA c 
JOIN (select COMMUNITY_AREA_NUMBER,COUNT(*) AS CRIME_COUNT from CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
HAVING COUNT(*) = (SELECT MAX(CRIME_COUNT) FROM (select COMMUNITY_AREA_NUMBER,COUNT(*) AS CRIME_COUNT
from CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER))) d on c.COMMUNITY_AREA_NUMBER=d.COMMUNITY_AREA_NUMBER"""

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, selectQuery)

result_dict = ibm_db.fetch_assoc(selectStmt)
while result_dict is not False:
    print(result_dict)
    result_dict = ibm_db.fetch_assoc(selectStmt)

{'COMMUNITY_AREA_NAME': 'Austin', 'COMMUNITY_AREA_NUMBER': 25, 'CRIME_COUNT': 43}
